In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


# Saving and loading models

 ## Coding tutorials
 #### [1. Saving and loading model weights](#coding_tutorial_1)
 #### [2. Model saving criteria](#coding_tutorial_2)
 #### [3. Saving the entire model](#coding_tutorial_3)
 #### [4. Loading pre-trained Keras models](#coding_tutorial_4)
 #### [5. Tensorflow Hub modules](#coding_tutorial_5)

***
<a id="coding_tutorial_1"></a>
## Saving and loading model weights

#### Load and inspect CIFAR-10 dataset

The CIFAR-10 dataset consists of, in total, 60000 color images, each with one of 10 labels: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck. For an introduction and a download, see [this link](https://www.cs.toronto.edu/~kriz/cifar.html).

In [2]:
# Import the CIFAR-10 dataset and rescale the pixel values

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Use smaller subset -- speeds things up
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:1000]
y_test = y_test[:1000]

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
# Plot the first 10 CIFAR-10 images

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    ax[i].set_axis_off()
    ax[i].imshow(x_train[i])

#### Introduce two useful functions

In [4]:
# Introduce function to test model accuracy

def get_test_accuracy(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [5]:
# Introduce function that creates a new instance of a simple CNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

def get_new_model():
    model = Sequential([
        Conv2D(filters=16, input_shape=(32, 32, 3), kernel_size=(3, 3), 
               activation='relu', name='conv_1'),
        Conv2D(filters=8, kernel_size=(3, 3), activation='relu', name='conv_2'),
        MaxPooling2D(pool_size=(4, 4), name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32, activation='relu', name='dense_1'),
        Dense(units=10, activation='softmax', name='dense_2')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

#### Create simple convolutional neural network classifier

In [6]:
# Create an instance of the model and show model summary

model=get_new_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 28, 28, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 7, 7, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 392)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                12576     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 14,514
Trainable params: 14,514
Non-trainable params: 0
____________________________________________________

In [ ]:
# Test accuracy of the untrained model, around 10% (random)



#### Train model with checkpoints

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
# Create Tensorflow checkpoint object

checkpoint_5000_path= \
'model_checkpoints_5000/checkpoint_{epoch:02d}_{batch:04d}'
checkpoint_5000=ModelCheckpoint(filepath=checkpoint_5000_path,save_weights_only=True, save_freq=5000, verbose=1)

In [10]:
# Fit model, with simple checkpoint which saves (and overwrites) model weights every epoch
model=get_new_model()
model.fit(x=x_train,y=y_train,epochs=3,validation_data=(x_test,y_test),batch_size=10,callbacks=[checkpoint_5000])

Train on 10000 samples, validate on 1000 samples
Epoch 1/3
 4990/10000 [=============>................] - ETA: 30s - loss: 2.1078 - accuracy: 0.2238
Epoch 00001: saving model to model_checkpoints_5000/checkpoint_01_0499
 9980/10000 [============================>.] - ETA: 0s - loss: 1.9673 - accuracy: 0.2742
Epoch 00001: saving model to model_checkpoints_5000/checkpoint_01_0999
10000/10000 [==============================] - 62s 6ms/sample - loss: 1.9662 - accuracy: 0.2744 - val_loss: 1.6963 - val_accuracy: 0.3890
Epoch 2/3
 4990/10000 [=============>................] - ETA: 28s - loss: 1.6543 - accuracy: 0.3872
Epoch 00002: saving model to model_checkpoints_5000/checkpoint_02_0499
 9990/10000 [============================>.] - ETA: 0s - loss: 1.6005 - accuracy: 0.4121
Epoch 00002: saving model to model_checkpoints_5000/checkpoint_02_0999
10000/10000 [==============================] - 61s 6ms/sample - loss: 1.6002 - accuracy: 0.4120 - val_loss: 1.6147 - val_accuracy: 0.4190
Epoch 3/3
 49

In [11]:
# Have a look at what the checkpoint creates

!ls -lh model_checkpoints_5000

total 1.1M
-rw-r--r-- 1 jovyan users   93 Dec 17 02:55 checkpoint
-rw-r--r-- 1 jovyan users 174K Dec 17 02:53 checkpoint_01_0499.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:53 checkpoint_01_0499.index
-rw-r--r-- 1 jovyan users 174K Dec 17 02:53 checkpoint_01_0999.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:53 checkpoint_01_0999.index
-rw-r--r-- 1 jovyan users 174K Dec 17 02:54 checkpoint_02_0499.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:54 checkpoint_02_0499.index
-rw-r--r-- 1 jovyan users 174K Dec 17 02:54 checkpoint_02_0999.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:54 checkpoint_02_0999.index
-rw-r--r-- 1 jovyan users 174K Dec 17 02:55 checkpoint_03_0499.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:55 checkpoint_03_0499.index
-rw-r--r-- 1 jovyan users 174K Dec 17 02:55 checkpoint_03_0999.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 17 02:55 checkpoint_03_0999.index


In [12]:
checkpoint_best_path='model_checkpoints_best/checkpoint'
checkpoint_best=ModelCheckpoint(filepath=checkpoint_best_path,
                               save_Weights_only=True, save_freq='epoch',monitor='val_accuracy',save_best_only=True,verbose=1)

In [17]:
# Evaluate the performance of the trained model
history=model.fit(x=x_train,y=y_train,epochs=10,validation_data=(x_test,y_test),batch_size=10,callbacks=[checkpoint_best])


Train on 10000 samples, validate on 1000 samples
Epoch 1/10
 9990/10000 [============================>.] - ETA: 0s - loss: 0.6306 - accuracy: 0.7748
Epoch 00001: val_accuracy did not improve from 0.60100
10000/10000 [==============================] - 60s 6ms/sample - loss: 0.6307 - accuracy: 0.7748 - val_loss: 1.4440 - val_accuracy: 0.5840
Epoch 2/10
 9980/10000 [============================>.] - ETA: 0s - loss: 0.6248 - accuracy: 0.7786
Epoch 00002: val_accuracy did not improve from 0.60100
10000/10000 [==============================] - 59s 6ms/sample - loss: 0.6261 - accuracy: 0.7783 - val_loss: 1.5465 - val_accuracy: 0.5520
Epoch 3/10
 9980/10000 [============================>.] - ETA: 0s - loss: 0.6112 - accuracy: 0.7831
Epoch 00003: val_accuracy did not improve from 0.60100
10000/10000 [==============================] - 59s 6ms/sample - loss: 0.6113 - accuracy: 0.7831 - val_loss: 1.5873 - val_accuracy: 0.5600
Epoch 4/10
 9980/10000 [============================>.] - ETA: 0s - loss

In [21]:
!ls -lh checkpoint_best_path

ls: cannot access 'checkpoint_best_path': No such file or directory


#### Create new model, load weights

In [19]:
# Create a new instance of the (initialised) model, accuracy around 10% again

new_model=get_new_model()
new_model.load_weights(checkpoint_best_path)
get_test_accuracy(new_model,x_test,y_test)

OSError: Unable to open file (file read failed: time = Fri Dec 17 04:37:36 2021
, filename = 'model_checkpoints_best/checkpoint', file descriptor = 45, errno = 21, error message = 'Is a directory', buf = 0x7ffeff3db5b0, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)

In [ ]:
# Load weights -- accuracy is the same as the trained model



#### Clear directory

In [ ]:
! rm -r model_checkpoints

***
<a id="coding_tutorial_2"></a>
## Model saving criteria

#### Create more customised checkpoint

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [13]:
# Create Tensorflow checkpoint object with epoch and batch details

checkpoint_5000_path= \

checkpoint_5000=ModelCheckpoint(filepath=checkpoint_5000_path,
                               save_weights_only=True,
                               save_frequency=5000,
                               verbose=1)

SyntaxError: invalid syntax (<ipython-input-13-6a67a0b0a15b>, line 4)

In [14]:
# Create and fit model with checkpoint
model.fit(x=x_train,y=y_train,epochs=3,validation_data=(x_test,y_test),
          batch_size=10,
          callbacks=[checkpoint_5000]
         )


Train on 10000 samples, validate on 1000 samples
Epoch 1/3
 4980/10000 [=============>................] - ETA: 29s - loss: 1.2144 - accuracy: 0.5691
Epoch 00001: saving model to model_checkpoints_5000/checkpoint_01_0499
 9980/10000 [============================>.] - ETA: 0s - loss: 1.2107 - accuracy: 0.5685
Epoch 00001: saving model to model_checkpoints_5000/checkpoint_01_0999
10000/10000 [==============================] - 60s 6ms/sample - loss: 1.2107 - accuracy: 0.5685 - val_loss: 1.2484 - val_accuracy: 0.5490
Epoch 2/3
 4990/10000 [=============>................] - ETA: 28s - loss: 1.1515 - accuracy: 0.5916
Epoch 00002: saving model to model_checkpoints_5000/checkpoint_02_0499
 9990/10000 [============================>.] - ETA: 0s - loss: 1.1593 - accuracy: 0.5924
Epoch 00002: saving model to model_checkpoints_5000/checkpoint_02_0999
10000/10000 [==============================] - 59s 6ms/sample - loss: 1.1590 - accuracy: 0.5924 - val_loss: 1.2206 - val_accuracy: 0.5670
Epoch 3/3
 49

In [ ]:
# Have a look at what the checkpoint creates



#### Work with model saving criteria

In [ ]:
# Use tiny training and test set -- will overfit!

x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]

In [ ]:
# Create a new instance of untrained model



In [ ]:
# Create Tensorflow checkpoint object which monitors the validation accuracy



In [ ]:
# Fit the model and save only the weights with the highest validation accuracy



In [ ]:
# Plot training and testing curves

import pandas as pd

df = pd.DataFrame(history.history)
df.plot(y=['accuracy', 'val_accuracy'])

In [ ]:
# Inspect the checkpoint directory



In [ ]:
# Create a new model with the saved weights



#### Clear directory

In [ ]:
! rm -r model_checkpoints_5000 model_checkpoints_best

***
<a id="coding_tutorial_3"></a>
## Saving the entire model

#### Create checkpoint that saves whole model, not just weights

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Create Tensorflow checkpoint object



In [ ]:
# Create and fit model with checkpoint



#### Inspect what the checkpoint has created

In [ ]:
# Have a look at what the checkpoint creates



In [ ]:
# Enter variables directory



In [ ]:
# Get the model's test accuracy



#### Create new model from scratch

In [ ]:
# Delete model



In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Reload model from scratch



#### Use the .h5 format to save model

In [ ]:
# Save the model in .h5 format



In [ ]:
# Inspect .h5 file



In [ ]:
# Delete model



In [ ]:
# Reload model from scratch



#### Clear directory

In [ ]:
! rm -r model_checkpoints
! rm my_model.h5

***
<a id="coding_tutorial_4"></a>
## Loading pre-trained Keras models

#### Import and build Keras ResNet50 model

Today we'll be using the ResNet50 model designed by a team at Microsoft Research, available through Keras applications. Please see the description on the [Keras applications page](https://keras.io/applications/#resnet) for details. If you continue using it, please cite it properly! The paper it comes from is:

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun. "Deep Residual Learning for Image Recognition", 2015.

This model takes a long time to download on the Coursera platform, so it is pre-downloaded in your workspace and saved in Keras HDF5 format. If you want to import it on your personal machine, use the following code:

```python
from tensorflow.keras.applications import ResNet50
model = ResNet50(weights='imagenet')
```

In this coding tutorial, you will instead load the model directly from disk.

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Build Keras ResNet50 model



#### Import and preprocess 3 sample images

In [ ]:
# Import 3 sample ImageNet images

from tensorflow.keras.preprocessing.image import load_img

lemon_img = load_img('data/lemon.jpg', target_size=(224, 224))
viaduct_img = load_img('data/viaduct.jpg', target_size=(224, 224))
water_tower_img = load_img('data/water_tower.jpg', target_size=(224, 224))

#### Use ResNet50 model to classify images

In [ ]:
# Useful function: presents top 5 predictions and probabilities

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd

def get_top_5_predictions(img):
    x = img_to_array(img)[np.newaxis, ...]
    x = preprocess_input(x)
    preds = decode_predictions(model.predict(x), top=5)
    top_preds = pd.DataFrame(columns=['prediction', 'probability'],
                             index=np.arange(5)+1)
    for i in range(5):
        top_preds.loc[i+1, 'prediction'] = preds[0][i][1]
        top_preds.loc[i+1, 'probability'] = preds[0][i][2] 
    return top_preds

##### Image 1: lemon

In [ ]:
# Display image



In [ ]:
# Display top 5 predictions



##### Image 2: viaduct

In [ ]:
# Display image



In [ ]:
# Display top 5 predictions



##### Image 3: water tower

In [ ]:
# Display image



In [ ]:
# Display top 5 predictions



***
<a id="coding_tutorial_5"></a>
## Tensorflow Hub modules

#### Import and build Tensorflow Hub MobileNet v1 model

Today we'll be using Google's MobileNet v1 model, available on Tensorflow Hub. Please see the description on the [Tensorflow Hub page](https://tfhub.dev/google/imagenet/mobilenet_v1_050_160/classification/4) for details on it's architecture, how it's trained, and the reference. If you continue using it, please cite it properly! The paper it comes from is:

Andrew G. Howard, Menglong Zhu, Bo Chen, Dmitry Kalenichenko, Weijun Wang, Tobias Weyand, Marco Andreetto, Hartwig Adam: "MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications", 2017.

This model takes a long time to download on the Coursera platform, so it is pre-downloaded in your workspace and saved in Tensorflow SavedModel format. If you want to import it on your personal machine, use the following code:

```python
module_url = "https://tfhub.dev/google/imagenet/mobilenet_v1_050_160/classification/4"
model = Sequential([hub.KerasLayer(module_url)])
model.build(input_shape=[None, 160, 160, 3])
```

In this coding tutorial, you will instead load the model directly from disk.

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras.models import load_model

In [ ]:
# Build Google's Mobilenet v1 model



#### Use MobileNet model to classify images

In [ ]:
# Import and preprocess 3 sample ImageNet images

from tensorflow.keras.preprocessing.image import load_img

lemon_img = load_img("data/lemon.jpg", target_size=(160, 160))
viaduct_img = load_img("data/viaduct.jpg", target_size=(160, 160))
water_tower_img = load_img("data/water_tower.jpg", target_size=(160, 160))

In [ ]:
# Read in categories text file

with open('data/imagenet_categories.txt') as txt_file:
    categories = txt_file.read().splitlines()

In [ ]:
# Useful function: presents top 5 predictions

import pandas as pd

def get_top_5_predictions(img):
    x = img_to_array(img)[np.newaxis, ...] / 255.0
    preds = model.predict(x)
    top_preds = pd.DataFrame(columns=['prediction'],
                             index=np.arange(5)+1)
    sorted_index = np.argsort(-preds[0])
    for i in range(5):
        ith_pred = categories[sorted_index[i]]
        top_preds.loc[i+1, 'prediction'] = ith_pred
            
    return top_preds

##### Image 1: lemon

##### Image 2: viaduct

##### Image 3: water tower